In [1]:
import os

import logging
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.datasets
import torchvision.transforms as transforms
import torchvision



from mean_teacher import datasets, architectures
from mean_teacher.utils import *

from IPython.display import clear_output

device = "cuda" if torch.cuda.is_available() else "cpu"

import matplotlib.pyplot as plt

LOG = logging.getLogger('main')
NO_LABEL = -1
to_image = transforms.ToPILImage()

In [2]:
dataset_config = datasets.__dict__['sslMini']()

In [3]:
def load_weights(model_arch, pretrained_model_path, state_dict, cuda=True):
        # Load pretrained model
        pretrained_model = torch.load(f=pretrained_model_path, map_location="cuda" if cuda else "cpu")

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in pretrained_model[state_dict].items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v

        # Load pre-trained weights in current model
        with torch.no_grad():
            model_arch.load_state_dict(new_state_dict, strict=True)

        # Debug loading
        #print('Parameters found in pretrained model:')
        pretrained_layers = new_state_dict.keys()
        #for l in pretrained_layers:
        #    print('\t' + l)
        #print('')

        for name, module in model_arch.state_dict().items():
            if name in pretrained_layers:
                assert torch.equal(new_state_dict[name].cpu(), module.cpu())
                #print('{} have been loaded correctly in current model.'.format(name))
            else:
                raise ValueError("state_dict() keys do not match")
                
        return model_arch


In [4]:
BATCH_SIZE = 32

evaldir = "/scratch/ijh216/ssl_mini/supervised/val"

eval_loader = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(evaldir, dataset_config['eval_transformation']),
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              num_workers=2,
                                              #pin_memory=True,
                                              drop_last=False)


In [5]:
pretrained_model_path = "/scratch/ijh216/ssl/ssl_shake_mini_augment/2019-05-06_18-04-18/10/transient/checkpoint.325.ckpt"
log = torch.load(f=pretrained_model_path, map_location="cuda" if False else "cpu")

In [6]:
log.keys()

dict_keys(['epoch', 'global_step', 'arch', 'state_dict', 'ema_state_dict', 'best_prec1', 'optimizer'])

In [7]:
model_dir = "/scratch/ijh216/ssl/ssl_shake_mini_augment/2019-05-06_18-04-18/10/transient/checkpoint.325.ckpt" 
model = architectures.__dict__['cifar_shakeshake26']().to(device)
model = load_weights(model, model_dir, state_dict="ema_state_dict", cuda=False)

In [8]:
def validate(eval_loader, model, log, global_step, epoch, device=device):
    class_criterion = nn.CrossEntropyLoss(reduction='sum', ignore_index=NO_LABEL).to(device)
    meters = AverageMeterSet()
    model.eval()

    end = time.time()
    for i, (img, target) in enumerate(eval_loader):
        meters.update('data_time', time.time() - end)
        
        img_var, target_var = img.to(device), target.to(device)
        
        minibatch_size = len(target_var)
        labeled_minibatch_size = target_var.detach().ne(NO_LABEL).sum()
        meters.update('labeled_minibatch_size', labeled_minibatch_size)

        # compute output
        output1, output2 = model(img_var)
        softmax1, softmax2 = F.softmax(output1, dim=1), F.softmax(output2, dim=1)
        class_loss = class_criterion(output1, target_var) / minibatch_size

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output1, target_var, topk=(1,5))
        meters.update('class_loss', class_loss.item(), labeled_minibatch_size)
        meters.update('top1', acc1, labeled_minibatch_size)
        meters.update('error1', 100.0 - acc1, labeled_minibatch_size)
        meters.update('top5', acc5, labeled_minibatch_size)
        meters.update('error5', 100.0 - acc5, labeled_minibatch_size)

        # measure elapsed time
        meters.update('batch_time', time.time() - end)
        end = time.time()
        if i % 10 == 0:
            LOG.info(
                'Test: [{0}/{1}]\t'
                'Time {meters[batch_time]:.3f}\t'
                'Data {meters[data_time]:.3f}\t'
                'Class {meters[class_loss]:.4f}\t'
                'Acc@1 {meters[top1]:.3f}\t'
                'Acc@5 {meters[top5]:.3f}'.format(
                    i, len(eval_loader), meters=meters))
            
        if i == 10:
            break

    LOG.info(' * Acc@1 {top1.avg:.3f}\Acc@5 {top5.avg:.3f}'
          .format(top1=meters['top1'], top5=meters['top5']))

    return meters['top1'].avg

In [13]:
res = validate(eval_loader, model, LOG, 325, 325, device)

In [10]:
best_acc1 = 0
global best_acc1

In [11]:
best_acc1

0

In [14]:
best_acc1 = log['best_prec1']

In [22]:
float(res) > float(best_acc1)

True

In [23]:
float(res)

38.0

In [21]:
res

tensor(38)

In [19]:
float(best_acc1)

0.0

In [30]:
float(res) > log['best_prec1'].item()

True

In [32]:
max(float(res), log['best_prec1'].item())

33.0

In [12]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy@k for the specified values of k"""
    maxk = max(topk)
    labeled_minibatch_size = max(target.ne(NO_LABEL).sum(), 1e-8).item()

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True).item()
        res.append(correct_k * (100.0 / labeled_minibatch_size))
    return res

In [65]:
model_dir = "/scratch/ijh216/ssl/ssl_shake_mini_augment/2019-05-06_18-04-18/10/transient/checkpoint.325.ckpt" 
model = architectures.__dict__['cifar_shakeshake26']().to(device)
model = load_weights(model, model_dir, state_dict="ema_state_dict", cuda=True)

n_samples = 0.
n_correct_top_1 = 0
n_correct_top_k = 0

for i, (img, target) in enumerate(eval_loader):
    img, target = img.to(device), target.to(device)
    n_samples += BATCH_SIZE

        # Forward
    output = model(img)[0]

        # Top 1 accuracy
    pred_top_1 = torch.topk(output, k=1, dim=1)[1]
    n_correct_top_1 += pred_top_1.eq(target.view_as(pred_top_1)).int().sum().item()

        # Top k accuracy
    pred_top_k = torch.topk(output, k=5, dim=1)[1]
    target_top_k = target.view(-1, 1).expand(BATCH_SIZE, 5)
    n_correct_top_k += pred_top_k.eq(target_top_k).int().sum().item()
    
    if i % 100 == 0:
        print("******************************")
        print("Acc@1", n_correct_top_1/n_samples)
        print("Acc@5", n_correct_top_k/n_samples)
        print("******************************")
    
    # Accuracy
top_1_acc = n_correct_top_1/n_samples
top_k_acc = n_correct_top_k/n_samples

print("******************************")
print("Acc@1", top_1_acc)
print("Acc@5", top_k_acc)
print("******************************")

******************************
Acc@1 0.40625
Acc@5 0.640625
******************************
******************************
Acc@1 0.29718440594059403
Acc@5 0.5383663366336634
******************************
******************************
Acc@1 0.3023165422885572
Acc@5 0.5359141791044776
******************************
******************************
Acc@1 0.30632267441860467
Acc@5 0.5371677740863787
******************************
******************************
Acc@1 0.303927680798005
Acc@5 0.5353413341645885
******************************
******************************
Acc@1 0.3029565868263473
Acc@5 0.5362400199600799
******************************
******************************
Acc@1 0.30355657237936773
Acc@5 0.5358257071547421
******************************
******************************
Acc@1 0.3045203281027104
Acc@5 0.5385164051355207
******************************
******************************
Acc@1 0.304229088639201
Acc@5 0.5373946629213483
******************************
************

In [11]:
for j, i in enumerate(os.listdir("/scratch/ijh216/ssl2"+'/unsupervised')):
    print(j, len(os.listdir("/scratch/ijh216/ssl2"+'/unsupervised/'+i)))

0 100
1 100
2 100
3 100
4 100
5 100
6 100
7 100
8 100
9 100
10 100
11 100
12 100
13 100
14 100
15 100
16 100
17 100
18 100
19 100
20 100
21 100
22 100
23 100
24 100
25 100
26 100
27 100
28 100
29 100
30 100
31 100
32 100
33 100
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100
60 100
61 100
62 100
63 100
64 100
65 100
66 100
67 100
68 100
69 100
70 100
71 100
72 100
73 100
74 100
75 100
76 100
77 100
78 100
79 100
80 100
81 100
82 100
83 100
84 100
85 100
86 100
87 100
88 100
89 100
90 100
91 100
92 100
93 100
94 100
95 100
96 100
97 100
98 100
99 100
100 100
101 100
102 100
103 100
104 100
105 100
106 100
107 100
108 100
109 100
110 100
111 100
112 100
113 100
114 100
115 100
116 100
117 100
118 100
119 100
120 100
121 100
122 100
123 100
124 100
125 100
126 100
127 100
128 100
129 100
130 100
131 100
132 100
133 100
134 100
135 100
136 100
137 100
138 10

In [40]:
img_dir = eval_loader.dataset.imgs[0][0]
img = eval_loader.dataset[0][0].unsqueeze(0).to(device)

In [49]:
labels = []

for ckpt in range(0, 325, 5):

    model_dir = "/scratch/ijh216/ssl/ssl_shake_mini_augment/2019-05-06_18-04-18/10/transient/checkpoint.{}.ckpt".format(ckpt) 
    model = architectures.__dict__['cifar_shakeshake26']().to(device)
    model = load_weights(model, model_dir, state_dict="ema_state_dict", cuda=False)


    output = F.softmax(model(img)[0], dim=1)
    labels.append(output)

labels = torch.cat(labels)    

In [52]:
labels = torch.cat(labels) 

In [58]:
torch.cat((labels, labels)).size()

torch.Size([130, 1000])

In [46]:
labels[0].size()

torch.Size([1, 1000])